In [186]:
import string
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import  stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('paraphrase-distilroberta-base-v1')



In [187]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

**preprocessing**

In [188]:
data = pd.read_csv('./Statedriven.csv', encoding='unicode-escape')
data

,Requirement with Standard Syntax,sentence1,sentence2,similarity_score,Similarity,New Requirement
0,"user register system, system shall allow user ...",user register system,system shall allow user select username passw...,0.395485,1.0,"Whenuser register system,then system shall al..."
1,"game numbrp resolution, game shall playable",game numbrp resolution,game shall playable,0.358248,1.0,"Whengame numbrp resolution,then game shall pl..."
2,"user access his/her account, system shall allo...",user access his/her account,system shall allow user enter password username.,0.410697,2.0,"Whenuser access his/her account,then system s..."
3,"user play tournament, system shall allow user ...",user play tournament,system shall allow user create team tournament.,0.548651,3.0,"Whenuser play tournament,then system shall al..."
4,"user participate respective tournament, system...",user participate respective tournament,system shall allow user select respective teams.,0.445075,4.0,"Whenuser participate respective tournament,the..."
5,"player participate tournaments, system shall a...",player participate tournaments,system shall allow play one tournament,0.477092,3.0,"Whenplayer participate tournaments,then syste..."
6,"match played, system shall allow store informa...",match played,system shall allow store information match pl...,0.256208,5.0,"Whenmatch played,then system shall allow stor..."
7,"match played, system shall allow stor informat...",match played,system shall allow stor information team matc...,0.334374,2.0,"Whenmatch played,then system shall allow stor..."
8,"match played respective touenament, system sha...",match played respective touenament,system shall store every outcome match.,0.248345,6.0,"Whenmatch played respective touenament,then s..."
9,bowler specific bowler id system shall allow s...,bowler specific bowler id system shall allow s...,bowler specific bowler id system shall allow s...,1.000000,7.0,Whenbowler specific bowler id system shall all...


In [189]:
data.columns
# printing all the columns headers

Index(['Requirement with Standard Syntax', 'sentence1', 'sentence2',
       'similarity_score', 'Similarity', 'New Requirement'],
      dtype='object')

In [190]:
data = data[['Requirement with Standard Syntax']]
# selecting only the required columns

In [191]:
data
# printing the data

,Requirement with Standard Syntax
0,"user register system, system shall allow user ..."
1,"game numbrp resolution, game shall playable"
2,"user access his/her account, system shall allo..."
3,"user play tournament, system shall allow user ..."
4,"user participate respective tournament, system..."
5,"player participate tournaments, system shall a..."
6,"match played, system shall allow store informa..."
7,"match played, system shall allow stor informat..."
8,"match played respective touenament, system sha..."
9,bowler specific bowler id system shall allow s...


In [192]:
data.dropna(inplace=True)
# dropping the rows with null values in any column


In [193]:

# create a new column with null values
data['sentence1'] = data['Requirement with Standard Syntax'].isnull()
data['sentence2'] = data['Requirement with Standard Syntax'].isnull()
data['similarity_score'] = data['Requirement with Standard Syntax'].isnull()

data


,Requirement with Standard Syntax,sentence1,sentence2,similarity_score
0,"user register system, system shall allow user ...",False,False,False
1,"game numbrp resolution, game shall playable",False,False,False
2,"user access his/her account, system shall allo...",False,False,False
3,"user play tournament, system shall allow user ...",False,False,False
4,"user participate respective tournament, system...",False,False,False
5,"player participate tournaments, system shall a...",False,False,False
6,"match played, system shall allow store informa...",False,False,False
7,"match played, system shall allow stor informat...",False,False,False
8,"match played respective touenament, system sha...",False,False,False
9,bowler specific bowler id system shall allow s...,False,False,False


In [194]:
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.lower()
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'£|\$', 'dollers')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumber')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].str.replace(r'\d+(\.\d+)?', 'numbr')
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].apply(lambda x: ' '.join(term for term in x.split() if term not in string.punctuation))
stop_words = set(stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
lemmatizer = WordNetLemmatizer()
data['Requirement with Standard Syntax'] = data['Requirement with Standard Syntax'].apply(lambda x: ' '.join(lemmatizer.lemmatize(t) for t in x.split()))


In [196]:

def calculate_similarity_score(sentence1, sentence2, model):
    # Generate embeddings for both sentences
    embeddings = model.encode([sentence1, sentence2])

    # Compute cosine similarity between the embeddings
    similarity = cosine_similarity(embeddings[0].reshape(1, -1), embeddings[1].reshape(1, -1))

    # Extract the similarity score (a value between -1 and 1)
    similarity_score = similarity.item()

    return similarity_score

def compareWithall(sentence1):
    sim=0
    rowCount = 0
    #compare the sentence with all the sentences in sentence2 column
    for index, row in data.iterrows():
        similarity= calculate_similarity_score(sentence1, row['sentence2'], model)
        if similarity > 0.3:
            sim = sim + 1
            data.at[index, 'Similarity'] = sim
            rowCount = rowCount + 1

         


# iterate through the rows of the data frame Requirement with Standard Syntax
for index, row in data.iterrows():
    #split the sentence if it contains a full a comma
    if ',' in row['Requirement with Standard Syntax']:
        sentence1 = row['Requirement with Standard Syntax'].split(',')[0]
        sentence2 = row['Requirement with Standard Syntax'].split(',')[1]
        data.at[index, 'sentence1'] = sentence1
        data.at[index, 'sentence2'] = sentence2
        data.at[index, 'similarity_score'] = calculate_similarity_score(sentence1, sentence2, model)
    else:
        data.at[index, 'sentence1'] = row['Requirement with Standard Syntax']
        data.at[index, 'sentence2'] = row['Requirement with Standard Syntax']
        data.at[index, 'similarity_score'] = calculate_similarity_score(row['Requirement with Standard Syntax'], row['Requirement with Standard Syntax'], model)

for index, row in data.iterrows():
    compareWithall(row['sentence1'])





#writing the data to a csv file
data.to_csv('Statedriven.csv', index=False)


In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# punkt is a pre-trained tokenizer for english language and averaged_perceptron_tagger is a pre-trained part-of-speech tagger for english language

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WB\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from collections import Counter
from nltk import word_tokenize, pos_tag, pos_tag_sents
texts = data['Requirement with Standard Syntax'].tolist()
tagged_texts = pos_tag_sents(map(word_tokenize, texts))


data['POS'] = tagged_texts
data

,Requirement with Standard Syntax,sentence1,sentence2,similarity_score,Similarity,New Requirement,POS
0,"user register system, system shall allow user ...",user register system,system shall allow user select username passw...,0.395485,1.0,"Whenuser register system,then system shall al...","[(user, RB), (register, NN), (system, NN), (,,..."
1,"game numbrp resolution, game shall playable",game numbrp resolution,game shall playable,0.358248,1.0,"Whengame numbrp resolution,then game shall pl...","[(game, NN), (numbrp, DT), (resolution, NN), (..."
2,"user access his/her account, system shall allo...",user access his/her account,system shall allow user enter password username.,0.410697,2.0,"Whenuser access his/her account,then system s...","[(user, JJ), (access, NN), (his/her, NN), (acc..."
3,"user play tournament, system shall allow user ...",user play tournament,system shall allow user create team tournament.,0.548651,3.0,"Whenuser play tournament,then system shall al...","[(user, JJ), (play, NN), (tournament, NN), (,,..."
4,"user participate respective tournament, system...",user participate respective tournament,system shall allow user select respective teams.,0.445075,4.0,"Whenuser participate respective tournament,the...","[(user, JJ), (participate, VB), (respective, J..."
5,"player participate tournaments, system shall a...",player participate tournaments,system shall allow play one tournament,0.477092,3.0,"Whenplayer participate tournaments,then syste...","[(player, NN), (participate, NN), (tournaments..."
6,"match played, system shall allow store informa...",match played,system shall allow store information match pl...,0.256208,5.0,"Whenmatch played,then system shall allow stor...","[(match, NN), (played, VBD), (,, ,), (system, ..."
7,"match played, system shall allow stor informat...",match played,system shall allow stor information team matc...,0.334374,2.0,"Whenmatch played,then system shall allow stor...","[(match, NN), (played, VBD), (,, ,), (system, ..."
8,"match played respective touenament, system sha...",match played respective touenament,system shall store every outcome match.,0.248345,6.0,"Whenmatch played respective touenament,then s...","[(match, NN), (played, VBD), (respective, JJ),..."
9,bowler specific bowler id system shall allow s...,bowler specific bowler id system shall allow s...,bowler specific bowler id system shall allow s...,1.0,7.0,Whenbowler specific bowler id system shall all...,"[(bowler, NN), (specific, JJ), (bowler, NN), (..."
